<a href="https://colab.research.google.com/github/tldtldcpfl/JoChaeRin/blob/master/cancer_%EB%A1%9C%EC%A7%93%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

설명
- dataset : cancer genetic mutations (암유전적 변이)

- training_text 데이터는 gene, variation, class로 구성돼어있다. 

- model:  로지스틱 회귀 모델 (ml알고리즘)을 통해 데이터의 class를 분류한다.


- evaluation:  모델의 분류 정확도는 54%로 낮은 성능을 보였다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")
import nltk
from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df_tab = pd.read_csv('/content/training_variants')

In [4]:
df_tab.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [5]:
df_tab.shape

(3321, 4)

In [6]:
# Count of Unique Values of Gene
df_tab["Gene"].value_counts()

BRCA1     264
TP53      163
EGFR      141
PTEN      126
BRCA2     125
         ... 
CTLA4       1
KDM5A       1
RAD51B      1
PIK3R3      1
STAG2       1
Name: Gene, Length: 264, dtype: int64

In [7]:
# Count of Unique Values of Variation
df_tab["Variation"].value_counts()

Truncating Mutations    93
Deletion                74
Amplification           71
Fusions                 34
Overexpression           6
                        ..
S252W                    1
M541L                    1
ETV6-PDGFRB Fusion       1
N319D                    1
S170R                    1
Name: Variation, Length: 2996, dtype: int64

In [8]:
# Count of Unique Values of Class
df_tab["Class"].value_counts()

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64

In [9]:
df_tab.isnull().sum()

ID           0
Gene         0
Variation    0
Class        0
dtype: int64

Reading Text Dataset and EDA

In [10]:
df_text =pd.read_csv("/content/training_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
df_text.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [11]:
df_text.shape

(518, 2)

Merge Datasets

In [12]:
df_merge = pd.merge(df_tab,df_text, on="ID",how="left")
df_merge.head()

,ID,Gene,Variation,Class,TEXT
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [13]:
# lowercase column names
df_merge.columns = df_merge.columns.str.lower()

전처리

Clean Text Column

In [14]:
df_merge.isnull().sum()

id              0
gene            0
variation       0
class           0
text         2803
dtype: int64

In [15]:
df_merge[df_merge.isnull().any(axis=1)]

,id,gene,variation,class,text
518,518,TP53,P151A,4,NaN
519,519,TP53,R273G,1,NaN
520,520,TP53,V197E,1,NaN
521,521,TP53,E286K,4,NaN
522,522,TP53,K120M,4,NaN
...,...,...,...,...,...
3316,3316,RUNX1,D171N,4,NaN
3317,3317,RUNX1,A122*,1,NaN
3318,3318,RUNX1,Fusions,1,NaN
3319,3319,RUNX1,R80C,4,NaN


In [16]:
# Fill NaN values with Gene & Variation 
df_merge.loc[df_merge["text"].isnull(),"text"] = df_merge["gene"] + " " + df_merge["variation"]

In [17]:
df_merge.isnull().sum()

id           0
gene         0
variation    0
class        0
text         0
dtype: int64

In [18]:
# lowercase 
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(i.lower() for i in x.split()))
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclin-dependent kinases (cdks) regulate a var...
1,1,CBL,W802*,2,abstract background non-small cell lung cancer...
2,2,CBL,Q249E,2,abstract background non-small cell lung cancer...
3,3,CBL,N454D,3,recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,oncogenic mutations in the monomeric casitas b...


In [19]:
# remove punctuations
df_merge["text"] = df_merge["text"].str.replace("[^\w\s]","")
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclindependent kinases cdks regulate a variet...
1,1,CBL,W802*,2,abstract background nonsmall cell lung cancer ...
2,2,CBL,Q249E,2,abstract background nonsmall cell lung cancer ...
3,3,CBL,N454D,3,recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,oncogenic mutations in the monomeric casitas b...


In [20]:
# remove numeric values
df_merge["text"] = df_merge["text"].str.replace("\d","")
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclindependent kinases cdks regulate a variet...
1,1,CBL,W802*,2,abstract background nonsmall cell lung cancer ...
2,2,CBL,Q249E,2,abstract background nonsmall cell lung cancer ...
3,3,CBL,N454D,3,recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,oncogenic mutations in the monomeric casitas b...


In [21]:
# remove stopwords: the,a,an etc.
nltk.download("stopwords")
from nltk.corpus import stopwords
sw = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(i for i in x.split() if i not in sw))
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclindependent kinases cdks regulate variety ...
1,1,CBL,W802*,2,abstract background nonsmall cell lung cancer ...
2,2,CBL,Q249E,2,abstract background nonsmall cell lung cancer ...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas blineage...


토크나이징

- 문장을 단어로 나누기

In [24]:
nltk.download("punkt")
import textblob
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
# an example:
TextBlob(df_merge["text"][1]).words

WordList(['abstract', 'background', 'nonsmall', 'cell', 'lung', 'cancer', 'nsclc', 'heterogeneous', 'group', 'disorders', 'number', 'genetic', 'proteomic', 'alterations', 'ccbl', 'e', 'ubiquitin', 'ligase', 'adaptor', 'molecule', 'important', 'normal', 'homeostasis', 'cancer', 'determined', 'genetic', 'variations', 'ccbl', 'relationship', 'receptor', 'tyrosine', 'kinases', 'egfr', 'met', 'functionality', 'nsclc', 'methods', 'findings', 'using', 'archival', 'formalinfixed', 'paraffin', 'embedded', 'ffpe', 'extracted', 'genomic', 'dna', 'show', 'ccbl', 'mutations', 'occur', 'somatic', 'fashion', 'lung', 'cancers', 'ccbl', 'mutations', 'mutually', 'exclusive', 'met', 'egfr', 'mutations', 'however', 'independent', 'p', 'kras', 'mutations', 'normaltumor', 'pairwise', 'analysis', 'significant', 'loss', 'heterozygosity', 'loh', 'ccbl', 'locus', 'n', 'none', 'samples', 'revealed', 'mutation', 'remaining', 'copy', 'ccbl', 'ccbl', 'loh', 'also', 'positively', 'correlated', 'egfr', 'met', 'mutati

In [27]:
df_merge["text"] = df_merge["text"].apply(lambda x: TextBlob(x).words)
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,"[cyclindependent, kinases, cdks, regulate, var..."
1,1,CBL,W802*,2,"[abstract, background, nonsmall, cell, lung, c..."
2,2,CBL,Q249E,2,"[abstract, background, nonsmall, cell, lung, c..."
3,3,CBL,N454D,3,"[recent, evidence, demonstrated, acquired, uni..."
4,4,CBL,L399V,4,"[oncogenic, mutations, monomeric, casitas, bli..."


lemmetization 

- 예) '아름다운' -> '아름답다'

In [28]:
from textblob import Word
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [29]:
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(Word(word).lemmatize() for word in x))
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclindependent kinase cdks regulate variety f...
1,1,CBL,W802*,2,abstract background nonsmall cell lung cancer ...
2,2,CBL,Q249E,2,abstract background nonsmall cell lung cancer ...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutation monomeric casitas blineage ...


In [30]:
df_merge.to_pickle("clean_text.pkl")

데이터셋 나누기

In [31]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_merge[["gene","variation","text"]],
                                                                    df_merge["class"], 
                                                                    random_state = 7,
                                                                    stratify=df_merge["class"],
                                                                    test_size=0.2)

In [32]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((2656, 3), (665, 3), (2656,), (665,))

In [33]:
train_x.head()

,gene,variation,text
2874,BRCA2,D1420Y,brca dy
3242,DDR2,G253C,ddr gc
2942,GNA11,Q209L,gna ql
2347,JAK2,I1018W,jak iw
1043,TSC2,L493P,tsc lp


TF-IDF    (단어빈도-역문서빈도)

- 여러 문서군에서 어떤 단어가 특정 문서 내에서 얼마나 중요한지를 나타내는 통계적 수치

- 각 단어의 중요도를 가중치로 부여 

In [35]:
# tf-idf-word-vectorizer 정의

tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [36]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

['a__p', 'a_ginsga', 'a_ginsyvma', 'a_sinstla', 'a_vdupasv']

In [37]:
len(tf_idf_word_vectorizer.get_feature_names())

39840

In [38]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x.text)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x.text)

로지스틱 회귀 모델 정의

- 범주형 변수 예측 모델 --> 초평면으로 데이터 포인트의 범주를 분류 
- 다항로지스틱 회귀 모델은 입력 벡터 x를 k개의 각각의 클래스로 분류 
- 입력: x값 -> 출력: 0~1 사이의 확률값 

In [39]:
# 로지스틱 회귀 모델 (범주형 변수 예측 모델) 정의
loj = LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word, train_y)

In [40]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics

In [41]:
# 로지스틱 회귀 모델의 정확도를 평가해보자

accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy) #정확도가 굉장히 낮다

Count Vectors Doğruluk Oranı: 0.5218453188602442


In [42]:
train_x.head()

,gene,variation,text
2874,BRCA2,D1420Y,brca dy
3242,DDR2,G253C,ddr gc
2942,GNA11,Q209L,gna ql
2347,JAK2,I1018W,jak iw
1043,TSC2,L493P,tsc lp


In [43]:
train_y.head()

2874    6
3242    7
2942    7
2347    6
1043    7
Name: class, dtype: int64

In [44]:
test_x.head()

,gene,variation,text
2937,BTK,Y551F,btk yf
609,CDK4,Amplification,cdk amplification
2973,KIT,Y578C,kit yc
1302,MLH1,R100*,mlh r
814,ERCC2,P463L,ercc pl


In [45]:
df_merge.head()

,id,gene,variation,class,text
0,0,FAM58A,Truncating Mutations,1,cyclindependent kinase cdks regulate variety f...
1,1,CBL,W802*,2,abstract background nonsmall cell lung cancer ...
2,2,CBL,Q249E,2,abstract background nonsmall cell lung cancer ...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutation monomeric casitas blineage ...


In [46]:
df_merge.shape

(3321, 5)

In [47]:
df_merge.dtypes

id            int64
gene         object
variation    object
class         int64
text         object
dtype: object

In [48]:
gene_enc = pd.get_dummies(df_merge["gene"],drop_first=True)

In [49]:
variation_enc = pd.get_dummies(df_merge["variation"],drop_first=True)

In [50]:
df_enc = pd.concat([gene_enc,variation_enc],axis=1)

In [51]:
df_enc2 = pd.concat([df_merge["class"],df_enc],axis=1)
df_enc2.head()

,class,ACVR1,AGO2,AKT1,AKT2,AKT3,ALK,APC,AR,ARAF,ARID1A,ARID1B,ARID2,ARID5B,ASXL1,ASXL2,ATM,ATR,ATRX,AURKA,AURKB,AXIN1,AXL,B2M,BAP1,BARD1,BCL10,BCL2,BCL2L11,BCOR,BRAF,BRCA1,BRCA2,BRD4,BRIP1,BTK,CARD11,CARM1,CASP8,CBL,...,Y599_D600insGLYVDFREYEY,Y599_D600insPAPQIMSTSTLISENMNIA,Y599_D600insSTDNEYFYVDFREYEY,Y62C,Y63C,Y640F,Y646C,Y646F,Y646H,Y646N,Y646S,Y647C,Y64A,Y652H,Y65C,Y68D,Y68H,Y69H,Y772_A775dup,Y791F,Y801H,Y803N,Y806C,Y823D,Y835F,Y842C,Y846C,Y849C,Y849S,Y87C,Y87N,Y901C,Y931C,Y98H,Y98N,YAP1-FAM118B Fusion,YAP1-MAMLD1 Fusion,ZC3H7B-BCOR Fusion,ZNF198-FGFR1 Fusion,p61BRAF
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_enc2.drop("class",axis=1),
                                                                    df_enc2["class"], 
                                                                    random_state = 7,
                                                                    stratify=df_merge["class"],
                                                                    test_size=0.2)

In [53]:
loj = LogisticRegression()
loj_model = loj.fit(train_x, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           test_x, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.5428539122568973


In [54]:
tf_idf_word_vectorizer = TfidfVectorizer()
text_enc = tf_idf_word_vectorizer.fit_transform(df_merge.text)

In [55]:
text_enc_df = pd.DataFrame(text_enc.toarray())

In [56]:
df_enc3 = pd.concat([df_enc2,text_enc_df],axis=1)
df_enc3.head()

,class,ACVR1,AGO2,AKT1,AKT2,AKT3,ALK,APC,AR,ARAF,ARID1A,ARID1B,ARID2,ARID5B,ASXL1,ASXL2,ATM,ATR,ATRX,AURKA,AURKB,AXIN1,AXL,B2M,BAP1,BARD1,BCL10,BCL2,BCL2L11,BCOR,BRAF,BRCA1,BRCA2,BRD4,BRIP1,BTK,CARD11,CARM1,CASP8,CBL,...,44388,44389,44390,44391,44392,44393,44394,44395,44396,44397,44398,44399,44400,44401,44402,44403,44404,44405,44406,44407,44408,44409,44410,44411,44412,44413,44414,44415,44416,44417,44418,44419,44420,44421,44422,44423,44424,44425,44426,44427
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006945,0.0,0.0,0.015367,0.0,0.0,0.000000,0.0,0.0,0.0,0.020115,0.0,0.0,0.0,0.0,0.021385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.008364,0.0,0.0,0.000000,0.0,0.0,0.0,0.004379,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.007398,0.0,0.0,0.003837,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.003432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
